In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\79252\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import cv2
import math
import numpy as np
import torch

In [3]:

model = YOLO('yolov8n-seg.pt')  



In [4]:
PowderBlue =np.array([ 176, 224, 230 ])
SlateBlue= np.array([	106, 90, 205 ])
PeachPuff = np.array([255, 218, 185])
colors = [PowderBlue, SlateBlue,PeachPuff ]

In [7]:
color_flag = None

In [11]:

cap = cv2.VideoCapture(0)
cap.set(3, 640) 
cap.set(4, 480) 

while True:
    success, img = cap.read()
    results = model(img, stream=True, classes = 0)

    if color_flag!=None:
        for r in results:
            
            if r.masks == None:
                continue
            boxes = r.boxes.data
            masks = r.masks.data
            
            
            clss = boxes[:, 5]
    
            people_indices = torch.where(clss == 0)
            
            people_masks = masks[people_indices]
            people_mask = torch.any(people_masks, dim=0).int() 

            background_mask = 1 - people_mask

            channel1 = np.zeros((480,640))
            channel2 = np.zeros((480,640))
            final_mask = np.hstack((background_mask.numpy(), channel1, channel2))

            img[ background_mask.bool() ] = np.ones((1,1,1)) * colors[color_flag]
            print(background_mask.bool().shape)
    cv2.imshow('Custom background', img)
    
    
    key = cv2.waitKey(10)

    if key == ord('q'):
        break
    elif key == ord('0'):
        color_flag = None
    elif key == ord('1'):
        color_flag = 0
    elif key == ord('2'):
        color_flag = 1
    elif key == ord('3'):
        color_flag = 2
        
cap.release()
cv2.destroyAllWindows()

In [12]:
cap.release()
cv2.destroyAllWindows()